In [330]:
#TODO: ASK DAVID WHAT HE DID ABOUT IMAGINARY NUMBERS IN THE MEAN SPECTRAL ENERGY
import math
import scipy as sp
from scipy.stats import kurtosis, skew
import sklearn
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [304]:
data = pd.read_csv('Elbowing.txt', sep="\t", header=None)
#data.columns = ["R_BI", "R_TRI", "L_BI", "L_TRI", "R_THI", "R_HAM", "L_THI", "L_HAM"]
elbow = data[0:700]

data = pd.read_csv('Frontkicking.txt', sep="\t", header=None)
frontKick = data[0:700]

data = pd.read_csv('Hamering.txt', sep="\t", header=None)
hamering = data[0:700]

data = pd.read_csv('Headering.txt', sep="\t", header=None)
headering = data[0:700]

frames = [elbow, frontKick, hamering, headering]
#combined = pd.concat(frames)
#np.shape(combined)

In [305]:
def appendEnergyVector(dataFrame):
    #take in the dataFrame, set new columns as a^2 + b^2 for each pair of 2. 
    energyCols = [8, 9, 10, 11]
    i = 0
    for col in energyCols:
        dataFrame[col] = np.square(dataFrame[i])+np.square(dataFrame[i+1])
        i = i+2
    return dataFrame

elbow = appendEnergyVector(elbow)
frontKick = appendEnergyVector(frontKick)
hamering = appendEnergyVector(hamering)
headering = appendEnergyVector(headering)

C:\Users\andre\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [306]:
#Mean crossings
def meanCrossing(frame):
    centeredFrame = frame - np.mean(frame)
    return ((centeredFrame[0][:-1].values * centeredFrame[0][1:].values) < 0).sum()

In [307]:
#4 bin histogram: It's actually 4 features. The bins are range/4. Check frequency of each bin for each feature. 
def fourBin(frame):
    allBins = numpy.histogram(frame[0],bins=4)[0][:]
    for i in range(1,12):
        allBins = np.append(allBins, numpy.histogram(frame[i],bins=4)[0][:])
    return allBins

In [328]:
featMean = np.empty([4,12])
featVar = np.empty([4,12])
featStd = np.empty([4,12])
featMin = np.empty([4,12])
featMax = np.empty([4,12])
featSkew = np.empty([4,12])
featKurtosis = np.empty([4,12])
featMeanCrossing = np.empty([4,12])
featMeanSpectralEnergy = np.empty([4,12], dtype=complex)
featFourBin = np.empty([4,48])

i=0
for frame in frames:
    featMean[i][:] = np.mean(frame).values.reshape([1,12])
    featVar[i][:] = np.var(frame).values.reshape([1,12])
    featStd[i][:] = np.std(frame).values.reshape([1,12])
    featMin[i][:] = np.min(frame).values.reshape([1,12])
    featMax[i][:] = np.max(frame).values.reshape([1,12])
    featSkew[i][:] = skew(frame)
    featKurtosis[i][:] = kurtosis(frame)
    featMeanCrossing[i][:] = meanCrossing(frame)
    featMeanSpectralEnergy[i][:] = np.mean(np.square(np.fft.fft(frame)),0)
    featFourBin[i][:] = fourBin(frame)
    i = i+1

allFeats = featMean
features = [featVar, featStd, featMin, featMax, featSkew, featKurtosis, featMeanCrossing, featMeanSpectralEnergy, featFourBin]
for feature in features:
    allFeats = np.append(allFeats, feature, axis=1)

print(np.shape(allFeats)) #4 samples by (12 channels by 13 features) = 4 x 156
        

(4, 156)


In [338]:
#Create and train classifier
y = np.reshape([1,2,3,4],[4,1])
#Train classifier. 
clf = RandomForestClassifier(n_jobs=2,random_state=0) #n_jobs to run in parallel. 
#Random_state is the seed used for the random number generator. 
clf.fit(allFeats,y)

C:\Users\andre\Anaconda3\lib\site-packages\sklearn\utils\validation.py:433: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)
C:\Users\andre\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [309]:
#np.var(elbow)

In [310]:
#np.std(elbow)

In [311]:
#np.min(elbow)

In [312]:
#np.max(elbow)

In [313]:
#skew(elbow)

In [314]:
#kurtosis(elbow)

In [316]:
#Mean crossings
#centeredElbow = elbow - means
#((centeredElbow[0][:-1].values * centeredElbow[0][1:].values) < 0).sum()


In [317]:
#Mean spectral energy
#fourier = np.fft.fft(elbow)
#uSpectralEnergy = np.mean(np.square(fourier),0)
#print(uSpectralEnergy)

In [158]:
#Test classifier by passing in 700 new values from something, getting the features, then predicing. 
